# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [57]:
pokemon=pd.read_csv("../Datasets/Pokemon.csv")

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [11]:
# hypothesis 1: sum of all 6

q=pokemon["Total"]==pokemon["HP"]+pokemon["Attack"]+pokemon["Defense"]+pokemon["Sp. Atk"]+pokemon["Sp. Def"]+pokemon["Speed"]
q.unique()

#it seems it is the sum of all 6

array([ True])

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [18]:
pokemon["Type 1"].unique()


array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [19]:
pokemon["Type 2"].unique()  #they have the same types

array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [47]:
one_hot=pd.get_dummies(pokemon["Type 1"])
one_hot.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [60]:
# your code here
pokemon2=pd.get_dummies(pokemon, columns= ["Type 1","Type 2"])

#order is different but I guess it is the same?

In [61]:
pokemon2

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,1,Bulbasaur,318,45,49,49,65,65,45,1,...,0,0,0,0,0,1,0,0,0,0
1,2,Ivysaur,405,60,62,63,80,80,60,1,...,0,0,0,0,0,1,0,0,0,0
2,3,Venusaur,525,80,82,83,100,100,80,1,...,0,0,0,0,0,1,0,0,0,0
3,3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,...,0,0,0,0,0,1,0,0,0,0
4,4,Charmander,309,39,52,43,60,50,65,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,600,50,100,150,100,150,50,6,...,0,0,0,0,0,0,0,0,0,0
796,719,DiancieMega Diancie,700,50,160,110,160,110,110,6,...,0,0,0,0,0,0,0,0,0,0
797,720,HoopaHoopa Confined,600,80,110,60,150,130,70,6,...,1,0,0,0,0,0,0,0,0,0
798,720,HoopaHoopa Unbound,680,80,160,60,170,130,80,6,...,0,0,0,0,0,0,0,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [63]:
correlation=pokemon2.corr()

In [65]:
correlation.columns

Index(['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Generation', 'Legendary', 'Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon',
       'Type 1_Electric', 'Type 1_Fairy', 'Type 1_Fighting', 'Type 1_Fire',
       'Type 1_Flying', 'Type 1_Ghost', 'Type 1_Grass', 'Type 1_Ground',
       'Type 1_Ice', 'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic',
       'Type 1_Rock', 'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug',
       'Type 2_Dark', 'Type 2_Dragon', 'Type 2_Electric', 'Type 2_Fairy',
       'Type 2_Fighting', 'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ghost',
       'Type 2_Grass', 'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal',
       'Type 2_Poison', 'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel',
       'Type 2_Water'],
      dtype='object')

In [71]:
correlation.loc["Total","Type 1_Bug":"Type 2_Water"]

Type 1_Bug        -0.143957
Type 1_Dark        0.017818
Type 1_Dragon      0.196532
Type 1_Electric    0.016715
Type 1_Fairy      -0.026948
Type 1_Fighting   -0.029086
Type 1_Fire        0.050527
Type 1_Flying      0.029504
Type 1_Ghost       0.007594
Type 1_Grass      -0.036057
Type 1_Ground      0.004082
Type 1_Ice        -0.002412
Type 1_Normal     -0.104150
Type 1_Poison     -0.057123
Type 1_Psychic     0.094364
Type 1_Rock        0.037524
Type 1_Steel       0.082000
Type 1_Water      -0.015640
Type 2_Bug        -0.021375
Type 2_Dark        0.065844
Type 2_Dragon      0.115240
Type 2_Electric    0.014669
Type 2_Fairy      -0.024606
Type 2_Fighting    0.138726
Type 2_Fire        0.073234
Type 2_Flying      0.054048
Type 2_Ghost      -0.004885
Type 2_Grass      -0.039224
Type 2_Ground      0.016486
Type 2_Ice         0.100870
Type 2_Normal     -0.013956
Type 2_Poison     -0.067837
Type 2_Psychic     0.076054
Type 2_Rock       -0.000512
Type 2_Steel       0.070307
Type 2_Water      -0

In [73]:
correlation.loc["Total","Type 1_Bug":"Type 2_Water"].idxmax()

#it seems that the highest correlation is found in the Type 1_Dragon 

'Type 1_Dragon'

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [74]:
correlation.loc["Total","Type 2_Bug":"Type 2_Water"].idxmax()

'Type 2_Fighting'

In [93]:
Dragon=pokemon[pokemon["Type 1"]=="Dragon"]

In [103]:
Dragon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
159,147,Dratini,Dragon,NaN,300,41,64,45,50,50,50,1,False
160,148,Dragonair,Dragon,NaN,420,61,84,65,70,70,70,1,False
161,149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,1,False
365,334,Altaria,Dragon,Flying,490,75,70,90,70,105,80,3,False
366,334,AltariaMega Altaria,Dragon,Fairy,590,75,110,110,110,105,80,3,False
406,371,Bagon,Dragon,NaN,300,45,75,60,40,30,50,3,False
407,372,Shelgon,Dragon,NaN,420,65,95,100,60,50,50,3,False
408,373,Salamence,Dragon,Flying,600,95,135,80,110,80,100,3,False
409,373,SalamenceMega Salamence,Dragon,Flying,700,95,145,130,120,90,120,3,False
417,380,Latias,Dragon,Psychic,600,80,80,90,110,130,110,3,True


In [100]:
Dragon["Type 2"].value_counts()

Flying      6
Ground      5
Psychic     4
Ice         3
Fairy       1
Fire        1
Electric    1
Name: Type 2, dtype: int64

In [133]:
#Dragon[["Type 1", "Type 2", "Total"]].sort_values("Total",ascending=False)

In [111]:
#pokemon[["Type 1", "Type 2", "Total"]].sort_values("Total",ascending=False)

In [115]:
Dragon_dummies=pd.get_dummies(Dragon, columns= ["Type 1","Type 2"])

In [130]:
Dragon_dummies.corr().index

Index(['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Generation', 'Legendary', 'Type 1_Dragon', 'Type 2_Electric',
       'Type 2_Fairy', 'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ground',
       'Type 2_Ice', 'Type 2_Psychic'],
      dtype='object')

In [132]:
#I obtained the correlation between type 2 and total of thos pokemons which type 1 is Dragon

Dragon_dummies.corr().loc["Total","Type 2_Electric":"Type 2_Psychic"]

Type 2_Electric    0.161521
Type 2_Fairy       0.049240
Type 2_Fire        0.161521
Type 2_Flying      0.304104
Type 2_Ground     -0.085285
Type 2_Ice         0.304144
Type 2_Psychic     0.261146
Name: Total, dtype: float64

In [ ]:
#It seems that the best suited to be second option is first Ice and then flying with very little difference
#followed by Psychic